<a href="https://colab.research.google.com/github/blindblackcat/assets/blob/main/Homework001zoomcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import datetime
from datetime import date

In [ ]:
SP500ListLink="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
SP500=pd.read_html(SP500ListLink)


In [ ]:
SP500Stocks = SP500[0]
SP500Changes = SP500[1]

In [ ]:
SP500Stocks

In [ ]:
SP500Changes

In [ ]:
SP500Stocks["Date added"]=pd.to_datetime(SP500Stocks["Date added"])

In [ ]:
pt=pd.pivot_table(SP500Stocks,values="Security",index="Date added",aggfunc="count")

In [ ]:
yearly=pt.resample("YE").sum().sort_values(by="Security",ascending=False)

In [ ]:
from matplotlib import pyplot as plt
yearly.plot(kind='bar', figsize=(20, 10), title='Yearly Changes')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
pt.plot(kind='line', figsize=(20, 10), title='Yearly Changes')
plt.show()

In [ ]:
Top3 = yearly.nlargest(3,"Security")
Top3


In [ ]:
SP500Changes[('Date', 'Date')] = pd.to_datetime(SP500Changes[('Date', 'Date')])
mask = SP500Changes[('Date', 'Date')].dt.year.isin([2016, 2017])
filtered_df = SP500Changes.loc[mask]


In [ ]:
filtered_df



In [ ]:
!pip install bertopic sentence-transformers




In [ ]:
# Install dependencies if not done
# !pip install bertopic sentence-transformers umap-learn plotly

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import umap.umap_ as umap
import pandas as pd
import plotly.io as pio

# OPTIONAL: Display Plotly in browser
pio.renderers.default = "browser"

# Load and clean your data
texts = SP500Changes[('Reason', 'Reason')].dropna().astype(str).tolist()

# UMAP: better tuned for short text
umap_model = umap.UMAP(
    n_neighbors=3,
    n_components=2,
    min_dist=0.1,
    metric='cosine',
    random_state=42
)

# SentenceTransformer: good for short texts
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize BERTopic with embedding + umap
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    language="english",
    verbose=True
)

# Fit model
topics, probs = topic_model.fit_transform(texts)

# Get topic info
topic_info = topic_model.get_topic_info()
print(topic_info.head())

# Visualize
fig = topic_model.visualize_topics()
fig = topic_model.visualize_topics()
fig.write_html("topics_visualization.html")
from google.colab import files
files.download("topics_visualization.html")


**Answer to question 1:**

The Year with the most additions was 2017

D1:
* Low interest rates made it easy and cheap for companies and private equity firms to finance mergers and acquisitions.

* Private equity activity surged, leading to many public companies
being acquired and taken private, requiring replacements in the index.

* Tech and pharma companies had large cash reserves and used them to:



1. Acquire smaller companies
2. Spin off divisions
3. Restructure operations

*  Spin off divisions,Restructure operations,Anticipation of U.S. tax reform under the Trump administration led to a rush of corporate actions before policy changes took effect.

D2

* Market capitalization shifts triggered index adjustments, as companies grew or shrank relative to others.

* Structural changes to the S&P, like the creation of the Real Estate sector in 2016.

* These years were marked by unusually high corporate activity, especially in tech, pharma, and private equity  all contributing to the pike.

In [ ]:
SP500ComponentStocks.columns

In [ ]:
SP500ComponentStocks["TimeInIndex"]=pd.Timestamp.today()-SP500ComponentStocks["Date added"]
SP500ComponentStocks["TimeInIndex"]=SP500ComponentStocks["TimeInIndex"].dt.days/365 #Not 252 days since is datetime year, not financial.

In [ ]:
Y20securities=SP500ComponentStocks[SP500ComponentStocks["TimeInIndex"]>20].sort_values(by="TimeInIndex",ascending=0).reset_index(drop=1)

In [ ]:
Y20securities

#Clean The data:

1. understand why Date added before foundation date in some securities.

In [ ]:
SP500ComponentStocks["Founded_clean"] = SP500ComponentStocks["Founded"].astype(str).str.extract(r'(\d{4})')

SP500ComponentStocks["error"] = SP500ComponentStocks.apply(
    lambda row: (
        f"Error: added in {pd.to_datetime(row['Date added'], errors='coerce').year} "
        f"but founded in {int(row['Founded_clean'])}"
    )
    if pd.notna(row["Date added"])
       and pd.notna(row["Founded_clean"])
       and pd.to_datetime(row["Date added"], errors='coerce').year < int(row["Founded_clean"])
    else None,
    axis=1
)


In [ ]:
SP500ComponentStocks

In [ ]:
SP500ComponentStocks[SP500ComponentStocks["error"].notna()][["Security", "error"]].reset_index(drop=1)


In [ ]:
error_list = pd.DataFrame({
    "Security": [
        "AbbVie", "Altria", "Bank of America", "Bristol Myers Squibb", "Cigna", "Citigroup", "ConocoPhillips",
        "Coterra", "CSX Corporation", "CVS Health", "DTE Energy", "Elevance Health", "Exelon", "ExxonMobil",
        "JPMorgan Chase", "L3Harris", "Lockheed Martin", "Molson Coors Beverage Company", "NextEra Energy",
        "Northrop Grumman", "Paramount Global", "Simon Property Group", "Tapestry, Inc.", "Thermo Fisher Scientific",
        "TJX Companies", "Walgreens Boots Alliance"
    ],
    "error": [
        "Error: added in 2012 but founded in 2013", "Error: added in 1957 but founded in 1985",
        "Error: added in 1976 but founded in 1998", "Error: added in 1957 but founded in 1989",
        "Error: added in 1976 but founded in 1982", "Error: added in 1988 but founded in 1998",
        "Error: added in 1957 but founded in 2002", "Error: added in 2008 but founded in 2021",
        "Error: added in 1957 but founded in 1980", "Error: added in 1957 but founded in 1996",
        "Error: added in 1957 but founded in 1995", "Error: added in 2002 but founded in 2014",
        "Error: added in 1957 but founded in 2000", "Error: added in 1957 but founded in 1999",
        "Error: added in 1975 but founded in 2000", "Error: added in 2008 but founded in 2019",
        "Error: added in 1957 but founded in 1995", "Error: added in 1976 but founded in 2005",
        "Error: added in 1976 but founded in 1984", "Error: added in 1957 but founded in 1994",
        "Error: added in 1994 but founded in 2019", "Error: added in 2002 but founded in 2003",
        "Error: added in 2004 but founded in 2017", "Error: added in 2004 but founded in 2006",
        "Error: added in 1985 but founded in 1987", "Error: added in 1979 but founded in 2014"
    ]
})

# The corrected founded and S&P 500 added dates for these companies
corrections = pd.DataFrame({
    "Security": [
        "AbbVie", "Altria", "Bank of America", "Bristol Myers Squibb", "Cigna", "Citigroup", "ConocoPhillips",
        "Coterra", "CSX Corporation", "CVS Health", "DTE Energy", "Elevance Health", "Exelon", "ExxonMobil",
        "JPMorgan Chase", "L3Harris", "Lockheed Martin", "Molson Coors Beverage Company", "NextEra Energy",
        "Northrop Grumman", "Paramount Global", "Simon Property Group", "Tapestry, Inc.", "Thermo Fisher Scientific",
        "TJX Companies", "Walgreens Boots Alliance"
    ],
    "Correct Founded": [
        2012, 1985, 1998, 1989, 1982, 1998, 2002, 2021, 1980, 1963, 1995, 2014, 2000, 1999,
        2000, 2019, 1995, 2005, 1984, 1994, 2019, 2003, 2017, 2006, 1987, 2014
    ],
    "S&P 500 Added": [
        2012, 1957, 1976, 1957, 1976, 1988, 1957, 2008, 1957, 1997, 1957, 2002, 1957, 1957,
        1975, 2008, 1957, 1976, 1976, 1957, 1994, 2002, 2004, 2004, 1985, 1979
    ]
})





In [ ]:
# 1. Crear diccionarios de corrección
founded_dict = dict(zip(corrections['Security'], corrections['Correct Founded']))
added_dict = dict(zip(corrections['Security'], corrections['S&P 500 Added']))


SP500ComponentStocks['Founded'] = SP500ComponentStocks.apply(
    lambda row: founded_dict[row['Security']] if row['Security'] in founded_dict else row['Founded'],
    axis=1
)


SP500ComponentStocks['Date added'] = SP500ComponentStocks.apply(
    lambda row: added_dict[row['Security']] if row['Security'] in added_dict else row['Date added'],
    axis=1
)

SP500ComponentStocks['Year Added'] = pd.to_datetime(SP500ComponentStocks['Date added'], errors='coerce').dt.year



In [ ]:
current_year = datetime.now().year
SP500ComponentStocks["YearsInIndex"] = current_year - SP500ComponentStocks["Year Added"]

over_20_years_count = SP500ComponentStocks[SP500ComponentStocks["YearsInIndex"] >= 20]["Security"].nunique()

print("Número de compañías con más de 20 años en el S&P 500:", over_20_years_count)


229 companies have been in the index whether under their first foundation name or other.